# Stackoverflow data

In [2]:
%matplotlib inline

import os
import re
import nltk
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from hd_utils import monitor
from nltk.corpus import stopwords
from html.parser import HTMLParser
from sklearn.datasets.lfw import Bunch
from html_preprocessing import HTMLPreprocessor

## Args

In [3]:
args = Bunch()

## Read

In [4]:
args.sodata_kaggle_name = 'pythonquestions'

In [5]:
!kaggle datasets download -d stackoverflow/$args.sodata_kaggle_name

pythonquestions.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
_base_path = f'~/.kaggle/datasets/stackoverflow/{args.sodata_kaggle_name}'
questions_path = os.path.join(_base_path, 'Questions.csv')
tags_path = os.path.join(_base_path, 'Tags.csv')

In [7]:
%time qdf = pd.read_csv(questions_path, encoding='ISO-8859-1')
qdf.shape

CPU times: user 13.2 s, sys: 1.32 s, total: 14.5 s
Wall time: 28.6 s


(607282, 6)

In [8]:
qdf.head()

,Id,OwnerUserId,CreationDate,Score,Title,Body
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...
1,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...
2,535,154.0,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...
3,594,116.0,2008-08-03T01:15:08Z,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...
4,683,199.0,2008-08-03T13:19:16Z,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...


In [9]:
qdf.drop(['OwnerUserId', 'CreationDate', 'Score'], axis=1, inplace=True)
qdf.rename({c: c.lower() for c in qdf.columns}, axis=1, inplace=True)
qdf.head()

,id,title,body
0,469,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...
1,502,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...
2,535,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...
3,594,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...
4,683,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...


## Text

In [10]:
args.log_step = 5000

In [11]:
sample = qdf.body.sample().iloc[0]
sample, HTMLPreprocessor().fit_transform([sample])[0]

100%|██████████| 1/1 [00:00<00:00,  5.76it/s]


("<p>I have something like this,</p>\n\n<pre><code>newlist = []\n\nlist =['a','b','c','d','e']\n\ndict = {'a':['a','a1','a2','a3'],'b':['b','b1','b2','b3'],.....'e':['e','e1','e2','e3']\n   }   \n</code></pre>\n\n<p>I have tried like this,</p>\n\n<pre><code>for listval in list:\n    newlist.append(dict[listval]].values())\n</code></pre>\n\n<p>But am not getting expected result,my expectation is,</p>\n\n<pre><code>newlist = [['a','a1','a2','a3'],['b','b1','b2','b3'],....,['e','e1','e2','e3']]\n</code></pre>\n",
 'I have something like this, DCNL CODE DCNL I have tried like this, DCNL CODE DCNL But am not getting expected result,my expectation is, DCNL CODE')

In [ ]:
preprocessor = HTMLPreprocessor(monitor('[1.6.1] Bodies filtering', args.log_step, len(qdf)))
bodies = preprocessor.fit_transform(qdf.body)

  8%|▊         | 46372/607282 [01:41<20:23, 458.43it/s]

In [ ]:
text = [t + ' DCNL ' + b for t, b in zip(qdf.title, bodies)]
df = pd.DataFrame(text, columns=['text'])
df.head()

## Tags

In [ ]:
tdf = pd.read_csv(tags_path, encoding='latin1')
tdf.shape

In [ ]:
tdf.head()

In [1]:
pd.Data

NameError: name 'pd' is not defined

In [ ]:
for id_, tags in tdf.groupby('id'):
    pass

KeyboardInterrupt: 

In [71]:
# tdf.groupby('id').agg(lambda tags: ', '.join(sorted(tags))).head()

KeyboardInterrupt: 

In [17]:
for id_, tag in tdf.iterrows():
    pass

KeyboardInterrupt: 

## Save

In [ ]:
...

## End

In [ ]:
args